In [1]:
from moviepy.editor import VideoFileClip
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.metrics import mean_squared_error
from moviepy.editor import AudioFileClip
from datetime import datetime

In [2]:
def extract_audio(video_paths, audio_paths):
    for video_path, audio_path in zip(video_paths, audio_paths):
        
        # Открываем видеофайл
        video_clip = VideoFileClip(video_path)
        
        # Извлекаем аудиодорожку
        audio_clip = video_clip.audio
        
        # Сохраняем аудиодорожку в файл
        audio_clip.write_audiofile(audio_path)
        
        # Освобождаем ресурсы
        audio_clip.close()
        video_clip.close()

In [3]:
def create_spectrogram(audio_path, output_path, start_time, end_time):
    
    # Загрузка аудиофайла
    y, sr = librosa.load(audio_path, sr=None)

    # Определение временных рамок
    start_frame = int(start_time * sr)
    end_frame = int(end_time * sr)

    # Обрезка аудиофайла до указанных временных рамок
    y = y[start_frame:end_frame]

    # Создание спектрограммы
    D = librosa.amplitude_to_db(librosa.stft(y), ref=np.max)
    
    # Отображение спектрограммы без осей и цветовой шкалы
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(D, sr=sr, x_axis=None, y_axis=None)
    plt.axis('off')
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

In [4]:
def create_spectrograms(audio_files, spectrograms):
    for audio_file, spectrogram in zip(audio_files, spectrograms):
        create_spectrogram(audio_file, spectrogram, start_time, end_time)
        print(f"Спектрограмма {spectrogram} успешно создана и сохранена.")

In [5]:
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    
    # Загрузка изображения и изменение размера до целевого размера
    img = Image.open(image_path)
    img = img.resize(target_size)
    
    # Преобразование изображения в формат RGB, если оно имеет альфа-канал (RGBA)
    if img.mode == 'RGBA':
        img = img.convert('RGB')
    
    # Преобразование изображения в массив numpy
    img_array = np.array(img)
    # Добавление измерения для батча
    img_array = np.expand_dims(img_array, axis=0)
    # Предварительная обработка изображения, как это делается в модели ResNet50
    img_array = preprocess_input(img_array)
    return img_array

In [6]:
def extract_features(image_path, model):
    
    # Загрузка и предварительная обработка изображения
    img_array = load_and_preprocess_image(image_path)
    # Извлечение признаков из изображения при помощи модели
    features = model.predict(img_array)
    return features

In [7]:
video_files = ["VideoOriginal.mp4", "Video.mp4"]
audio_files = ["VideoOriginal.mp3", "Video.mp3"]
spectrograms = ["OriginalSpec.png", "Spect.png"]

In [8]:
extract_audio(video_files, audio_files)
print("Аудиодорожки успешно извлечены и сохранены.")

MoviePy - Writing audio in VideoOriginal.mp3


MoviePy - Done.
MoviePy - Writing audio in Video.mp3


MoviePy - Done.
Аудиодорожки успешно извлечены и сохранены.


In [9]:
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [10]:
start_time = 0
end_time = 2
threshold = 0.5

In [11]:
durations = [AudioFileClip(audio_files[0]).duration, AudioFileClip(audio_files[1]).duration]

In [12]:
print(durations)

[296.72, 274.0]


In [13]:
log_file = open('HistoryAudio.txt', 'w')

In [14]:
while (durations[0] >= end_time) & (durations[1] >= end_time):

    create_spectrograms(audio_files, spectrograms)
    
    original_spec_features = extract_features(spectrograms[0], resnet_model)
    spect_features = extract_features(spectrograms[1], resnet_model)

    mse_value = mean_squared_error(original_spec_features.flatten(), spect_features.flatten())

    log_file.write(f"{datetime.now()} - Среднеквадратическое отклонение (MSE): {mse_value}. Спектрограммы, в промежутке от {start_time} до {end_time} секунд видео, {'совпадают.' if mse_value < threshold else 'не совпадают.'}\n")
    
    start_time += 2
    end_time += 2

Спектрограмма OriginalSpec.png успешно создана и сохранена.
Спектрограмма Spect.png успешно создана и сохранена.
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Спектрограмма OriginalSpec.png успешно создана и сохранена.
Спектрограмма Spect.png успешно создана и сохранена.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Спектрограмма OriginalSpec.png успешно создана и сохранена.
Спектрограмма Spect.png успешно создана и сохранена.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Спектрограмма OriginalSpec.png успешно создана и сохранена.
Спектрограмма Spect.png успешно создана и сохранена.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Спектрограмма OriginalSpec.png успешно создана и сохранена.
Спектрограмма Spect.png успешно создана и сохранена.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Спектрограмма OriginalSpec.png успешно создана и сохранен

In [15]:
log_file.close()
print("Сравнение завершено. Результаты записаны в файл логов.")

Сравнение завершено. Результаты записаны в файл логов.
